## Total Cost Model Building and Exploration 

This notebook shows the process of building classification models for Total Costs. It shows the performance of models with different bin sizes, different models, and upsampling techniques. This notebook also contains an evaluation of model performance across sub-populations.

First, we will be importing all necessary libraries required for our Random Forest classification, and we will looking at various metrics including the accuracy, precision, and recall.

In [24]:
#Import all needed models
import sys
import platform
plat = platform.system()
if plat == 'Windows':
    sys.path.insert(0, '..\src\helpers')
    sys.path.insert(0, '..\src\models')
elif plat =='Linux' or plat=='Darwin':
    sys.path.insert(0, '../src/helpers')
    sys.path.insert(0, '../src/models')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.metrics import log_loss, f1_score, classification_report, make_scorer, precision_score, recall_score, accuracy_score, confusion_matrix, plot_confusion_matrix
import seaborn as sns
import imblearn
from imblearn.over_sampling import SMOTE

The below two cells must be run at all times in order to bring in the helper functions for the models and additonally clean data further for data modeling.

In [25]:
from data_cleaners import *
from model_building_helpers import *

In [26]:
#Load all patient csv (csv containing data for all the indications (COPD, Heart Failure, Schizophrenia, Knee Replacement, Kidney/UTI))
data_file_path = '../data'
all_patient_df = load_data('All', data_file_path)

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


###### Step 1 - Creating Bins for Total Cost

We will first create different bins for our classifier.

In [27]:
maxcost = all_patient_df['Total Costs'].max()
print("Maximum Total Cost:", maxcost)

#Bin the data by length of stay ranges
bins = [1, 5000, 10000, 15000, 20000, 30000, 50000, 1250000]
labels = ['1 - 5000', '5001 - 10000', '10001 - 15000', '15001 - 20000'
          , '20001 - 30000', '30001 - 50000', '50001 - 1250000']

all_patient_df_bins = all_patient_df.copy()
all_patient_df_bins['Total Costs Bin'] = pd.cut(x = all_patient_df_bins['Total Costs'], 
                                                bins = bins, labels = labels, include_lowest = True)
all_patient_df_bins.head(10)

Maximum Total Cost: 1159410


,Hospital Service Area,Hospital County,Permanent Facility Id,Age Group,Gender,Race,Ethnicity,Type of Admission,CCSR Diagnosis Code,CCSR Procedure Code,...,APR MDC Code,APR Severity of Illness Code,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Emergency Department Indicator,Total Costs,Total Charges,Length of Stay,Total Costs Bin
0,Hudson Valley,Westchester,1061,3,Male,White,Not Span/Hispanic,Elective,659,0,...,19,2,1,Medical,Medicare,0,40464,114168,21,30001 - 50000
1,Hudson Valley,Westchester,1061,4,Male,White,Spanish/Hispanic,Emergency,99,108,...,5,3,2,Medical,Medicaid,0,26046,58517,8,20001 - 30000
2,Hudson Valley,Westchester,1072,5,Male,Black/African American,Not Span/Hispanic,Emergency,108,193,...,5,2,3,Medical,Medicare,0,13980,26213,4,10001 - 15000
3,Hudson Valley,Sullivan,971,3,Female,White,Not Span/Hispanic,Emergency,659,0,...,19,2,2,Medical,Private Health Insurance,1,17794,39854,20,15001 - 20000
4,Hudson Valley,Sullivan,971,4,Female,White,Not Span/Hispanic,Emergency,108,0,...,5,3,3,Medical,Medicare,1,21015,69112,12,20001 - 30000
5,Hudson Valley,Sullivan,971,2,Male,Black/African American,Not Span/Hispanic,Emergency,659,0,...,19,2,1,Medical,Private Health Insurance,1,8232,18602,11,5001 - 10000
6,Hudson Valley,Sullivan,971,3,Male,Black/African American,Not Span/Hispanic,Emergency,659,0,...,19,2,1,Medical,Private Health Insurance,1,5528,12169,6,5001 - 10000
7,Hudson Valley,Sullivan,971,5,Female,White,Not Span/Hispanic,Emergency,159,0,...,11,3,3,Medical,Medicare,1,11323,39420,7,10001 - 15000
8,Hudson Valley,Sullivan,971,3,Male,White,Not Span/Hispanic,Emergency,659,0,...,19,2,2,Medical,Medicare,1,8750,19670,9,5001 - 10000
9,Hudson Valley,Sullivan,971,4,Female,White,Not Span/Hispanic,Emergency,159,0,...,11,3,3,Medical,Medicare,1,12787,40184,8,10001 - 15000


In [28]:
plt.figure(figsize = (8, 8))
plt.title('Count Plot for Cost Bins', fontsize=12, weight='bold', pad=30)
sns.countplot(data=all_patient_df_bins, y="Total Costs Bin", palette=['skyblue'])

<AxesSubplot:title={'center':'Count Plot for Cost Bins'}, xlabel='count', ylabel='Total Costs Bin'>

Error in callback <function flush_figures at 0x0000028A79182040> (for post_execute):


KeyboardInterrupt: 

###### Step 2 - Train model on Hist Gradient Boosting Classfier
This will help us with understanding the Accuracy and F1-scores of the model

In [ ]:
X, y, X_train, X_test, y_train, y_test = get_cost_train_test_data(all_patient_df_bins)

model = HistGradientBoostingClassifier()
model.fit(X_train, np.ravel(y_train))
y_pred = model.predict(X_test)
bin1_accuracy = accuracy_score(y_test, y_pred)
bin1_f1_score_macro = f1_score(y_test, y_pred, average='macro')
bin1_f1_score_weighted = f1_score(y_test, y_pred, average='weighted')
cf_matrix = confusion_matrix(y_test, y_pred)
print("Accuracy = ", bin1_accuracy, "\nF1 Macro Score = ", bin1_f1_score_macro, "\nF1 Weighted Score = ", bin1_f1_score_weighted)

In [ ]:
#Visualize the confusion matrix as a heart map
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix), index = [i for i in labels],
                  columns = [i for i in labels])
plt.figure(figsize = (12, 12))
plt.title('Confusion Matrix For HistGradientBoosting  Model', fontsize=17, weight='bold', pad=30)
sns.heatmap(df_cm, annot=True, fmt='.2%',  cmap='Blues')

In [ ]:
#Get Gradient Boost Classifier best parameters and best score by sub-population
#Experiment model performance by changing the parameters tested and interations for the RandomizedSearch
gb_scores = []
iterations = 10 #We found the iterations did not affect F1 macro scores much, so we used the default of 10

for drg in ['all', 194.0, 140.0, 750.0, 463.0, 302.0]:
    subpop_df = load_data(drg, data_file_path)
    subpop_df['Total Costs Bin'] = pd.cut(x = subpop_df['Total Costs'], bins = bins, labels = labels, 
                                          include_lowest = True)
    if len(subpop_df) > 30000:
        subpop_df = subpop_df.groupby('Total Costs Bin', group_keys=False).apply(
            lambda x: x.sample(int(np.rint(30000*len(x)/len(subpop_df))))).sample(frac=1).reset_index(drop=True)
        
    X, y, X_train, X_test, y_train, y_test = get_cost_train_test_data(subpop_df)

    random_grid = {'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
                   'learning_rate': [0.1 , 0.5 , 1],
                   'min_samples_leaf': [1, 2, 4],
                   'loss': ['log_loss', 'auto', 'binary_crossentropy','categorical_crossentropy']}
    
    best_params, best_score = get_best_gbc_params(X_train, y_train, random_grid, iterations)
    gb_scores.append({drg:{
            'best_params': best_params,
            'best_score': best_score
            }
        }
    )

In [ ]:
#Get LogisticRegression best parameters and best score by sub-population
#Experiment model performance by changing the parameters tested and interations for the RandomizedSearch
lr_scores = []
iterations = 10 #We found the iterations did not affect F1 macro scores much, so we used the default of 10

for drg in ['all', 194.0, 140.0, 750.0, 463.0, 302.0]:
    subpop_df = load_data(drg, data_file_path)
    subpop_df['Total Costs Bin'] = pd.cut(x = subpop_df['Total Costs'], bins = bins, 
                                          labels = labels, include_lowest = True)
    if len(subpop_df) > 30000:
        subpop_df = subpop_df.groupby('Total Costs Bin', group_keys=False).apply(
            lambda x: x.sample(int(np.rint(30000*len(x)/len(subpop_df)
                                          )
                                  )
                              )).sample(frac=1).reset_index(drop=True)
        
    X, y, X_train, X_test, y_train, y_test = get_cost_train_test_data(subpop_df)

    random_grid = {'C' : np.logspace(0, 4, num=10),
    'penalty' : ['l1', 'l2'],
    'solver' : ['liblinear'],
    'class_weight': ['balanced', None]}
    
    best_params, best_score = get_best_lr_params(X_train, y_train, random_grid, iterations)
    lr_scores.append({drg:{
            'best_params': best_params,
            'best_score': best_score
            }
        }
    )

In [ ]:
#Get Random Forest best parameters and best score by sub-population
#Experiment model performance by changing the parameters tested and interations for the RandomizedSearch
rf_scores = []
iterations = 10 #We found the iterations did not affect F1 macro scores much, so we used the default of 10

for drg in ['all', 194.0, 140.0, 750.0, 463.0, 302.0]:
    subpop_df = load_data(drg, data_file_path)
    subpop_df['Total Costs Bin'] = pd.cut(x = subpop_df['Total Costs'], bins = bins, 
                                          labels = labels, include_lowest = True)
    if len(subpop_df) > 30000:
        subpop_df = subpop_df.groupby('Total Costs Bin', group_keys=False).apply(
            lambda x: x.sample(int(np.rint(30000*len(x)/len(subpop_df)
                                          )
                                  )
                              )).sample(frac=1).reset_index(drop=True)
        
    X, y, X_train, X_test, y_train, y_test = get_cost_train_test_data(subpop_df)

    random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 800, num = 15)],
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False],
               'class_weight': ['balanced', 'balanced_subsample']}
    
    best_params, best_score = get_best_rf_params(X_train, y_train, random_grid, iterations)
    
    rf_scores.append({drg:{
            'best_params': best_params,
            'best_score': best_score
            }
        }
    )

In [ ]:
scores_gb_df = pd.DataFrame(columns=['model', 'subpopulation', 'best_params', 'best_scores'])
for item in gb_scores:
    for key, item in item.items():
        scores_gb_df = scores_gb_df.append({'model': 'HistGradientBoost', 'subpopulation': key, 'best_params': item['best_params'], 'best_scores': item['best_score']}, ignore_index=True)



In [ ]:
scores_lr_df = pd.DataFrame(columns=['model', 'subpopulation', 'best_params', 'best_scores'])
for item in lr_scores:
    for key, item in item.items():
        scores_lr_df = scores_lr_df.append({'model': 'LogRegression', 'subpopulation': key, 'best_params': item['best_params'], 'best_scores': item['best_score']}, ignore_index=True)



In [ ]:
scores_rf_df = pd.DataFrame(columns=['model', 'subpopulation', 'best_params', 'best_scores'])
for item in rf_scores:
    for key, item in item.items():
        scores_rf_df = scores_rf_df.append({'model': 'RandomForest', 'subpopulation': key, 'best_params': item['best_params'], 'best_scores': item['best_score']}, ignore_index=True)


In [ ]:
randomized_search_results = pd.concat([scores_gb_df,scores_lr_df, scores_rf_df])

In [ ]:
#Create visualization for each subpopulation for each model
fig, axes = plt.subplots(2, 3, figsize=(15, 10), sharey=True)

fig.subplots_adjust(hspace=0.45, wspace=0.25)


fig.suptitle('F1 - Macro Scores For HistGradientBoostedClassifier vs. RandomForestClassifier vs. LogisticRegression\n(Total Cost)', fontsize=17, weight='bold')


all_patients = randomized_search_results[randomized_search_results['subpopulation'] == 'all']
heart_patients = randomized_search_results[randomized_search_results['subpopulation'] == 194.0]
copd_patients = randomized_search_results[randomized_search_results['subpopulation'] == 140.0]
schizophrenia_patients = randomized_search_results[randomized_search_results['subpopulation'] == 750.0]
kidney_patients = randomized_search_results[randomized_search_results['subpopulation'] == 463.0]
knee_rep_patients = randomized_search_results[randomized_search_results['subpopulation'] == 302.0]

sns.barplot(ax=axes[0, 0], x=all_patients['model'], y=all_patients['best_scores'], palette=['orangered', 'skyblue','#5CED73'])
axes[0, 0].set_xlabel('Model Type', fontweight='bold', fontsize=14)
axes[0, 0].set_ylabel('F1 Score', fontweight='bold', fontsize=14)
axes[0, 0].bar_label(axes[0, 0].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[0, 0].set_xticklabels(labels=['Hist Gradient\nBoosted\nClassifier', 'Logistic\nRegression', 'Random Forest\nClassifier'], fontsize=14)
axes[0, 0].set_title('All Patients', fontweight='bold', fontsize=16)
current_values_y = axes[0, 0].get_yticks()
axes[0, 0].set_yticklabels([round(x, 2) for x in current_values_y], fontsize=14)

sns.barplot(ax=axes[0, 1], x=heart_patients['model'], y=heart_patients['best_scores'], palette=['orangered', 'skyblue','#5CED73'])
axes[0, 1].set_xlabel('Model Type', fontweight='bold', fontsize=14)
axes[0, 1].set_ylabel('F1 Score', fontweight='bold', fontsize=14)
axes[0, 1].bar_label(axes[0, 1].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[0,1].set_xticklabels(labels=['Hist Gradient\nBoosted\nClassifier', 'Logistic\nRegression', 'Random Forest\nClassifier'], fontsize=14)
axes[0, 1].set_title('Patients with DRG Code 194\n(Heart Failure)', fontweight='bold', fontsize=16)

sns.barplot(ax=axes[0, 2], x=copd_patients['model'], y=copd_patients['best_scores'], palette=['orangered', 'skyblue','#5CED73'])
axes[0, 2].set_xlabel('Model Type', fontweight='bold', fontsize=14)
axes[0, 2].set_ylabel('F1 Score', fontweight='bold', fontsize=14)
axes[0, 2].bar_label(axes[0, 2].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[0, 2].set_xticklabels(labels=['Hist Gradient\nBoosted\nClassifier', 'Logistic\nRegression', 'Random Forest\nClassifier'], fontsize=14)
axes[0, 2].set_title('Patients with DRG Code 140\n(COPD)', fontweight='bold', fontsize=16)

sns.barplot(ax=axes[1, 0], x=schizophrenia_patients['model'], y=schizophrenia_patients['best_scores'], palette=['orangered', 'skyblue','#5CED73'])
axes[1, 0].set_xlabel('Model Type', fontweight='bold', fontsize=12)
axes[1, 0].set_ylabel('F1 Score', fontweight='bold', fontsize=12)
axes[1, 0].bar_label(axes[1, 0].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[1, 0].set_xticklabels(labels=['Hist Gradient\nBoosted\nClassifier', 'Logistic\nRegression', 'Random Forest\nClassifier'],  fontsize=14)
axes[1, 0].set_title('Patients with DRG Code 750\n(Schizophrenia)', fontweight='bold', fontsize=16)
current_values_y = axes[1, 0].get_yticks()
axes[1, 0].set_yticklabels([round(x, 2) for x in current_values_y], fontsize=14)

sns.barplot(ax=axes[1, 1], x=kidney_patients['model'], y=kidney_patients['best_scores'], palette=['orangered', 'skyblue','#5CED73'])
axes[1, 1].set_xlabel('Model Type', fontweight='bold', fontsize=14)
axes[1, 1].set_ylabel('F1 Score', fontweight='bold', fontsize=14)
axes[1, 1].bar_label(axes[1, 1].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[1, 1].set_xticklabels(labels=['Hist Gradient\nBoosted\nClassifier', 'Logistic\nRegression', 'Random Forest\nClassifier'], fontsize=14)
axes[1, 1].set_title('Patients with DRG Code 463\n(Kidney/UTI)', fontweight='bold', fontsize=16)

sns.barplot(ax=axes[1, 2], x=knee_rep_patients['model'], y=knee_rep_patients['best_scores'], palette=['orangered', 'skyblue','#5CED73'])
axes[1, 2].set_xlabel('Model Type', fontsize=14, fontweight='bold')
axes[1, 2].set_ylabel('F1 Score', fontsize=14, fontweight='bold')
axes[1, 2].bar_label(axes[1, 2].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[1, 2].set_xticklabels(labels=['Hist Gradient\nBoosted\nClassifier', 'Logistic\nRegression', 'Random Forest\nClassifier'], fontsize=14)
axes[1, 2].set_title('Patients with DRG Code 302\n(Knee Joint Replacement)', fontweight='bold', fontsize=16)

axes[1, 2].set_ylim(0, 0.5)

plt.tight_layout(rect = [0, 0, 1, 0.988])
plt.savefig('data/visualizations/Cost_PerfomanceAcrossModels.png')

It looks like for most models, the HistGradientBoostingClassifier does almost the same as the Random Forest Classifier but does slightly better than the LogisticRegression model in for all subpopulations and all patients. Let's train each subpopulation on their best parameters for HistGradientBoostingClassifier model.

In [ ]:
final_model_scores = []

for drg in ['all', 194.0, 140.0, 750.0, 463.0, 302.0]:
    subpop_df = load_data(drg, data_file_path)
    subpop_df['Total Costs Bin'] = pd.cut(x = subpop_df['Total Costs'], bins = bins, 
                                          labels = labels, include_lowest = True)      
    X, y, X_train, X_test, y_train, y_test = get_cost_train_test_data(subpop_df)

    best_params = scores_gb_df.loc[(scores_gb_df['model']=='HistGradientBoost') & 
                                   (scores_gb_df['subpopulation']==drg)]['best_params'].values[0]
    model_scores = get_cost_model_scores(best_params, X_train, X_test, y_train, y_test)
    final_model_scores.append({
        'Population': drg,
        'Accuracy Score': model_scores[0],
        'F1_score_macro': model_scores[1],
        'F1_score_weighted': model_scores[2],
        'Model':'HistGradientBoost'
    })

final_results_df = pd.DataFrame(final_model_scores)

In [ ]:
#Visualize the results
#Build visualization comparing random forest model performance of all populations
colors = ['orangered' if (x == 'all') else 'skyblue' for x in final_results_df['Population'].values]

fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharey=False)
fig.subplots_adjust(hspace=0.45, wspace=0.25)
fig.suptitle('Comparison of Random Forest Model Performance Across Patient Populations\n(Total Cost)', fontsize=17, fontweight='bold')

sns.barplot(ax=axes[0], x=final_results_df['Population'], y=final_results_df['F1_score_macro'], palette=colors)
axes[0].set_xlabel('Subpopulation (DRG Code)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('F1 Macro Score', fontsize=12, fontweight='bold')
axes[0].bar_label(axes[0].containers[0], fmt='%.2f', padding=2, fontsize=12)
axes[0].set_xticklabels(labels=['All', 'Heart Failure\n(DRG 194)', 'COPD\n(DRG 140)', 'Schizophrenia\n(DRG 750)', 'Kidney\\UTI\n(DRG 463)', 'Knee Joint\nReplacement\n(DRG 302)'], fontsize=12)
axes[0].set_title('F1 Macro Score Across Populations', fontweight='bold', fontsize=14)
axes[0].set_ylim(0, 1)

sns.barplot(ax=axes[1], x=final_results_df['Population'], y=final_results_df['F1_score_weighted'], palette=colors)
axes[1].set_xlabel('Subpopulation (DRG Code)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('F1 Weighted Score', fontsize=12, fontweight='bold')
axes[1].bar_label(axes[1].containers[0], fmt='%.2f', fontsize=12)
axes[1].set_xticklabels(labels=['All', 'Heart Failure\n(DRG 194)', 'COPD\n(DRG 140)', 'Schizophrenia\n(DRG 750)', 'Kidney\\UTI\n(DRG 463)', 'Knee Joint\nReplacement\n(DRG 302)'], fontsize=12)
axes[1].set_title('F1 Weighted Score Across Populations', fontweight='bold', fontsize=14)
axes[1].set_ylim(0, 1)

sns.barplot(ax=axes[2], x=final_results_df['Population'], y=final_results_df['Accuracy Score'], palette=colors)
axes[2].set_xlabel('Subpopulation (DRG Code)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Accuracy Score', fontsize=12, fontweight='bold')
axes[2].bar_label(axes[2].containers[0], fmt='%.2f', fontsize=12)
axes[2].set_xticklabels(labels=['All', 'Heart Failure\n(DRG 194)', 'COPD\n(DRG 140)', 'Schizophrenia\n(DRG 750)', 'Kidney\\UTI\n(DRG 463)', 'Knee Joint\nReplacement\n(DRG 302)'], fontsize=12)
axes[2].set_title('Accuracy Score Across Populations', fontweight='bold', fontsize=14)
axes[2].set_ylim(0, 1)

plt.tight_layout(rect = [0, 0, 1, 0.988])

From the visualization above, it can be seen that for F1 macro scores the model built on all conditions of interest outperforms models trained only on patients with a specific DRG code with the exception of Knee replacement patients.
<br /><br />
Let's also compare performance with a dummy classifier which predicts the majority class every time and a dummy classifier which selects a class at random.

In [ ]:
#Get dummy model performance on all the population datasets
final_model_scores = []

#all population
for drg in ['all', 194.0, 140.0, 750.0, 463.0, 302.0]:
    print(drg)
    subpop_df = load_data(drg, data_file_path)
    subpop_df['Total Costs Bin'] = pd.cut(x = subpop_df['Total Costs'], bins = bins, 
                                          labels = labels, include_lowest = True) 
    X, y, X_train, X_test, y_train, y_test = get_cost_train_test_data(subpop_df)
    
    most_freq_dummy_scores = get_dummy_scores("most_frequent", X_train, X_test, y_train, y_test)

    final_model_scores.append({
        'Population': drg,
        'Model': 'DummyClassifier - Most Frequent',
        'Accuracy Score': most_freq_dummy_scores[0],
        'F1_score_macro': most_freq_dummy_scores[1],
        'F1_score_weighted': most_freq_dummy_scores[2]
    })

    random_dummy_scores = get_dummy_scores("uniform", X_train, X_test, y_train, y_test)

    final_model_scores.append({
        'Population': drg,
        'Model': 'DummyClassifier - Random',
        'Accuracy Score': random_dummy_scores[0],
        'F1_score_macro': random_dummy_scores[1],
        'F1_score_weighted': random_dummy_scores[2]
    })

In [ ]:
final_scores_df = pd.DataFrame(final_model_scores)
final_results_with_dummy_df = pd.concat([final_results_df, final_scores_df], ignore_index=True)

In [ ]:
#Plot dummy classifiers vs rf model performance for each population
fig, axes = plt.subplots(3, 1, figsize=(15, 15), sharey=False)
fig.subplots_adjust(hspace=0.45, wspace=0.25)
fig.suptitle('Scores of Trained Random Forest Model vs. Dummy Models By Subpopulation\n(Total Cost)', fontsize=17, fontweight='bold')

sns.barplot(ax=axes[0], data=final_results_with_dummy_df, x="Population", y="F1_score_macro", 
            hue="Model",palette= ['orangered','skyblue', '#5CED73'])
axes[0].set_xlabel('Subpopulation', fontsize=14, fontweight='bold')
axes[0].set_ylabel('F1 Macro Score', fontsize=14, fontweight='bold')
axes[0].set_xticklabels(labels=['All', 'Heart Failure\n(DRG 194)', 'COPD\n(DRG 140)', 'Schizophrenia\n(DRG 750)', 'Kidney\\UTI\n(DRG 463)', 'Knee Joint\nReplacement\n(DRG 302)'], fontsize=14)
axes[0].set_title('F1 Macro Scores', fontsize=14, fontweight='bold')
axes[0].set_ylim(0, 0.5)
current_values_y = axes[0].get_yticks()
axes[0].set_yticklabels([round(x, 2) for x in current_values_y], fontsize=14)
axes[0].legend(fontsize=12)

sns.barplot(ax=axes[1], data=final_results_with_dummy_df, x="Population", y="F1_score_weighted", 
            hue="Model",palette= ['orangered','#5CED73', 'skyblue', '#5CED73'])
axes[1].set_xlabel('Subpopulation', fontsize=14, fontweight='bold')
axes[1].set_ylabel('F1 Weighted Score', fontsize=14, fontweight='bold')
axes[1].set_xticklabels(labels=['All', 'Heart Failure\n(DRG 194)', 'COPD\n(DRG 140)', 'Schizophrenia\n(DRG 750)', 'Kidney\\UTI\n(DRG 463)', 'Knee Joint\nReplacement\n(DRG 302)'], fontsize=14)
axes[1].set_title('F1 Weighted Scores', fontsize=14, fontweight='bold')
current_values_y = axes[1].get_yticks()
axes[1].set_yticklabels([round(x, 2) for x in current_values_y], fontsize=14)
axes[1].legend(fontsize=12)

sns.barplot(ax=axes[2], data=final_results_with_dummy_df, x="Population", y="Accuracy Score", 
            hue="Model",palette= ['orangered', '#5CED73','skyblue', '#5CED73'])
axes[2].set_xlabel('Subpopulation', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Accuracy Score', fontsize=14, fontweight='bold')
axes[2].set_xticklabels(labels=['All', 'Heart Failure\n(DRG 194)', 'COPD\n(DRG 140)', 'Schizophrenia\n(DRG 750)', 'Kidney\\UTI\n(DRG 463)', 'Knee Joint\nReplacement\n(DRG 302)'], fontsize=14)
axes[2].set_title('Accuracy Scores', fontsize=14, fontweight='bold')
current_values_y = axes[2].get_yticks()
axes[2].set_yticklabels([round(x, 2) for x in current_values_y], fontsize=14)
axes[2].legend(fontsize=12)

for ax1 in [axes[0], axes[1], axes[2]]:
    for c in ax1.containers:
        # set the bar label
        ax1.bar_label(c, fmt='%.2f', fontsize=14)

plt.tight_layout(rect = [0, 0, 1, 0.988])

The models do outperform the dummy models, especially when comparing the F1 Macro score. However, overall we can see that the F1 macro scores are still fairly low for the trained models.

<br /><br />
As a next step, we will be performing Oversampling, because of the performance time of this, it has been commented out.

In [ ]:
## uncomment to run this cell

oversample_scores = []

oversample = SMOTE()
best_params = scores_gb_df.loc[(scores_gb_df['model']=='HistGradientBoost') & (scores_gb_df['subpopulation']=='all')]['best_params'].values[0]

subpop_df = load_data('all', data_file_path)
subpop_df['Total Costs Bin'] = pd.cut(x = subpop_df['Total Costs'], bins = bins, 
                                           labels = labels, include_lowest = True)

X, y, X_train, X_test, y_train, y_test = get_cost_train_test_data(subpop_df)

X_train_resample, y_train_resample = oversample.fit_resample(X_train, y_train)

oversampled_model = train_cost_model(best_params, X_train_resample, y_train_resample)

oversample_scores.append({
     'Population': 'all',
     'Model': 'Upsampled',
     'Accuracy Score': model_scores[0],
     'F1_score_macro': model_scores[1],
     'F1_score_weighted': model_scores[2]
})

final_results_df = final_results_df.append(oversample_scores)

In [ ]:
#Plot dummy classifiers vs rf model performance for each population
fig, axes = plt.subplots(1, 3, figsize=(15, 8), sharey=True)
fig.subplots_adjust(hspace=0.45, wspace=0.25)
fig.suptitle('Scores of Hist Gradient Boost Classifier Trained on Upsampled vs Original Data\n(Total Cost)', fontsize=16, fontweight='bold')

final_results_upsampled = final_results_df[final_results_df['Population'] == 'all']
sns.barplot(ax=axes[0], data=final_results_df, x="Model", y="F1_score_macro", palette= ['orangered', 'skyblue'], ci=None)
axes[0].set_xlabel('Model Type', fontsize=14, fontweight='bold')
axes[0].set_ylabel('F1 Macro Score', fontsize=14, fontweight='bold')
axes[0].bar_label(axes[0].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[0].set_title('F1 Macro Scores', fontsize=14, fontweight='bold')
axes[0].set_xticklabels(labels=['Hist Gradient\nBoost\nClassifier', 'Hist Gradient\nBoost\nClassifier\nUpsampled'], fontsize=14)

sns.barplot(ax=axes[1], data=final_results_df, x="Model", y="F1_score_weighted", palette= ['orangered', 'skyblue'], ci=None)
axes[1].set_xlabel('Model Type', fontsize=14, fontweight='bold')
axes[1].set_ylabel('F1 Weighted Score', fontsize=14, fontweight='bold')
axes[1].bar_label(axes[1].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[1].set_title('F1 Weighted Scores', fontsize=14, fontweight='bold')
axes[1].set_xticklabels(labels=['Hist Gradient\nBoost\nClassifier', 'Hist Gradient\nBoost\nClassifier\nUpsampled'], fontsize=14)


sns.barplot(ax=axes[2], data=final_results_df, x="Model", y="Accuracy Score", palette= ['orangered', 'skyblue'], ci=None)
axes[2].set_xlabel('Model Type', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Accuracy Score', fontsize=14, fontweight='bold')
axes[2].bar_label(axes[2].containers[0], fmt='%.2f', padding=2, fontsize=14)
axes[2].set_title('Accuracy Scores', fontsize=14, fontweight='bold')
axes[2].set_xticklabels(labels=['Hist Gradient\nBoost\nClassifier', 'Hist Gradient\nBoost\nClassifier\nUpsampled'], fontsize=14)